# Analysis

In [ ]:
import numpy as np
import pandas as pd

# 小樣本參數
T = 6  # t = 0 ~ 5（共6期），我們會建立 Qk_hat_2 ~ Qk_hat_5（共4期）
N = 1  #  item
simulation_times = 3
service_level = 0.95

# 生成 demand_df: 每一 row 是一個 item，每一 column 是一個時間點
np.random.seed(1)
demand_data = np.random.normal(loc=15, scale=3, size=(N, T)).round().astype(int)
demand_df = pd.DataFrame(demand_data)
print(f"demand_df.shape: {demand_df.shape}")
print(demand_df)

Qks = np.random.normal(loc=20, scale=5, size=(T - 2, N, simulation_times))
print(f"Qks.shape: {Qks.shape}")
print(Qks[:, :, :])


def make_Qk_hat_df_with_known_Qk(
    demand_df: pd.DataFrame,
    Qks: np.ndarray,
    service_level: float,
) -> pd.DataFrame:
    """
    計算每期的補貨量預估 (Qk_hat)。

    參數:
    ----------
    demand_df : pd.DataFrame
        每列為時間點 (t=0..T-1)，每欄為樣本 (N)。shape=(T, N)
    Qks : np.ndarray
        已知的模擬補貨量分布樣本，shape=(T-2, N, simulation_times)
    service_level : float
        服務水準 (例如 0.95 表示 95%)。

    回傳:
    ----------
    pd.DataFrame
        每列為樣本 (sample_0..sample_{N-1})，每欄為 Qk_hat_{2}..Qk_hat_{T-1}。

    範例 (Example):（只取一個 item 舉例）
    ----------
    假設:
        T = 6  # t = 0 ~ 5（共6期），我們會建立 Qk_hat_2 ~ Qk_hat_5（共4期）
        N = 1  #  item
        simulation_times = 3

        demand_df (shape 1×6):
            0   1   2   3   4   5
        0  20  13  13  12  18   8

        代表 T=1 需求是 20
        T=2 需求是 13
        ...

        Qks (shape 4×1×3): -> 這是其中一筆 item 的資料，包含
        [[[28.724, 16.194, 21.595]],
         [[18.753, 27.311,  9.699]],
         [[18.388, 18.080, 25.669]],
         [[14.501, 19.138, 15.611]]]

    計算過程:
        - t=2 (計算 Qk_hat_2):
          x_observed = [20]  → sum=20 -> 只取 t=1
          Qk_percentile = 第95百分位 ≈ 28.0112
          Qk_hat_2 = 20 + 28.0112 = 48.0112

        - t=3 (計算 Qk_hat_3):
          x_observed = [20,13] → sum=33 -> 只取 t=1,2
          Qk_percentile ≈ 26.4548
          Qk_hat_3 = 33 + 26.4548 = 59.4548

        - t=4 (計算 Qk_hat_4):
          x_observed = [20,13,13] → sum=46 -> 只取 t=1,2,3
          Qk_percentile ≈ 24.9408
          Qk_hat_4 = 46 + 24.9408 = 70.9408

        - t=5 (計算 Qk_hat_5):
          x_observed = [20,13,13,12] → sum=58 -> 只取 t=1,2,3,4
          Qk_percentile ≈ 18.7851
          Qk_hat_5 = 58 + 18.7851 = 76.7851

    最終輸出:
        Qk_hat DataFrame (1×4):
               Qk_hat_2   Qk_hat_3   Qk_hat_4   Qk_hat_5
              48.0112     59.4548     70.9408     76.7851
    """

    N, T = demand_df.shape
    Qk_hat_matrix = np.zeros((N, T - 2))

    for t_index in range(T - 2):  # 0～3
        Qks_t = Qks[t_index, :, :]  # shape: (3, simulation_times)

        for item_idx in range(N):

            # 取出單一 item 的所有 Qk，並且計算 Qk 的 service_level 分位數
            qk_dist = Qks_t[item_idx]  # shape: (simulation_times,)
            Qk_percentile = np.percentile(qk_dist, service_level * 100)

            # 現在是計算 t=k 的 Qk hat, 因此取出已經觀測到的值，是 1 ~ k-1 的真實值
            x_observed = demand_df.iloc[item_idx, : t_index + 1].values
            print(
                f"x_observed: {x_observed}, x_observed.sum: {x_observed.sum()}, Qk_percentile: {Qk_percentile}"
            )
            Qk_hat_matrix[item_idx, t_index] = x_observed.sum() + Qk_percentile

    time_columns = [f"Qk_hat_{t+2}" for t in range(T - 2)]
    return pd.DataFrame(Qk_hat_matrix, columns=time_columns)


# 執行
Qk_hat_df_example = make_Qk_hat_df_with_known_Qk(demand_df, Qks, service_level)
Qk_hat_df_example

demand_df.shape: (1, 6)
    0   1   2   3   4  5
0  20  13  13  12  18  8
Qks.shape: (4, 1, 3)
[[[28.72405882 16.1939655  21.59519548]]

 [[18.75314812 27.31053969  9.69929645]]

 [[18.38791398 18.07972823 25.66884721]]

 [[14.50054366 19.13785896 15.61070791]]]
x_observed: [20], x_observed.sum: 20, Qk_percentile: 28.01117248700271
x_observed: [20 13], x_observed.sum: 33, Qk_percentile: 26.454800528963677
x_observed: [20 13 13], x_observed.sum: 46, Qk_percentile: 24.940753888502712
x_observed: [20 13 13 12], x_observed.sum: 58, Qk_percentile: 18.785143857062355


,Qk_hat_2,Qk_hat_3,Qk_hat_4,Qk_hat_5
0,48.011172,59.454801,70.940754,76.785144


In [1]:
# 列出所有 csv
import os

# 目錄路徑
directory = "/Users/hanyuan/Github/Two-Phase-Newsvendor/results_0407"

# 列出所有 .csv 檔案
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# 顯示結果
for file in csv_files:
    print(file)

summary.csv
test_chunk20_lasso1_fold5_lst0.8.csv
test_chunk20_lasso1_fold5_lst0.9.csv
test_chunk30_lasso1_fold5_lst0.9.csv
test_chunk30_lasso1_fold5_lst0.8.csv
train_chunk30_lasso1_fold5_lst0.9.csv
train_chunk30_lasso1_fold5_lst0.8.csv
train_chunk20_lasso1_fold5_lst0.8.csv
train_chunk20_lasso1_fold5_lst0.9.csv


In [2]:
import os

# 資料夾路徑
directory = "/Users/hanyuan/Github/Two-Phase-Newsvendor/results_0407"

# 取得所有 .csv 檔案名稱
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# 用來存組合鍵
train_keys = set()
test_keys = set()

# 分析每個檔案
for filename in csv_files:
    if filename.startswith("train_"):
        key = filename.replace("train_", "")
        train_keys.add(key)
    elif filename.startswith("test_"):
        key = filename.replace("test_", "")
        test_keys.add(key)

# 找出只有在 train 有的組合
train_only = train_keys - test_keys

# 找出只有在 test 有的組合
test_only = test_keys - train_keys

# 找出同時有 train 和 test 的組合
matched = train_keys & test_keys

# 顯示結果
print("✅ Matched train-test pairs:")
for key in sorted(matched):
    print(f" - {key}")

print("\n❌ Only in train:")
for key in sorted(train_only):
    print(f" - {key}")

print("\n❌ Only in test:")
for key in sorted(test_only):
    print(f" - {key}")

✅ Matched train-test pairs:
 - chunk20_lasso1_fold5_lst0.8.csv
 - chunk20_lasso1_fold5_lst0.9.csv
 - chunk30_lasso1_fold5_lst0.8.csv
 - chunk30_lasso1_fold5_lst0.9.csv

❌ Only in train:

❌ Only in test:


In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 設定 seaborn 樣式
sns.set(style="whitegrid")

# 🔧 參數設定
input_dir = "/Users/hanyuan/Github/Two-Phase-Newsvendor/results_0407"
output_dir = os.path.join(input_dir, "plots")
os.makedirs(output_dir, exist_ok=True)  # 如果 plots 資料夾不存在就建立

# 🔍 取得所有 train/test 檔案
files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
train_files = {f.replace("train_", ""): f for f in files if f.startswith("train_")}
test_files = {f.replace("test_", ""): f for f in files if f.startswith("test_")}

# 🔗 找出交集組合鍵
common_keys = sorted(set(train_files.keys()) & set(test_files.keys()))

print(f"Found {len(common_keys)} matched pairs.")

# 🎯 對每組配對做分析和畫圖
for key in common_keys:
    print(f"-----------------------------------")
    print(f"key: {key}")
    print(f"train_files[key]: {train_files[key]}")
    print(f"test_files[key]: {test_files[key]}")

    train_path = os.path.join(input_dir, train_files[key])
    test_path = os.path.join(input_dir, test_files[key])

    # 讀取 CSV 檔案
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # train_df = train_df.drop(columns=["S12", "S15"])

    # 計算平均
    train_means = train_df.mean()
    test_means = test_df.mean()

    # baseline 和 theory 最佳
    baseline_train = train_means["baseline"]
    baseline_test = test_means["baseline"]
    theory_best_train = train_means["S14"]
    theory_best_test = test_means["S14"]

    # 百分比變化
    train_pct_base = (train_means - baseline_train) / baseline_train * 100
    test_pct_base = (test_means - baseline_test) / baseline_test * 100
    train_pct_theory = (train_means - theory_best_train) / theory_best_train * 100
    test_pct_theory = (test_means - theory_best_test) / theory_best_test * 100

    # 整理成 DataFrame
    avg_df = pd.DataFrame(
        {
            "Method": train_means.index,
            "Train": train_means.values,
            "Test": test_means.values,
            "Train_%_Base": train_pct_base.values,
            "Test_%_Base": test_pct_base.values,
            "Train_%_Theory": train_pct_theory.values,
            "Test_%_Theory": test_pct_theory.values,
        }
    )

    # 長格式轉換
    avg_df_melted = avg_df.melt(
        id_vars=[
            "Method",
            "Train_%_Base",
            "Test_%_Base",
            "Train_%_Theory",
            "Test_%_Theory",
        ],
        value_vars=["Train", "Test"],
        var_name="Dataset",
        value_name="Average Profit",
    )

    # 畫圖
    plt.figure(figsize=(15, 9))
    ax = sns.barplot(x="Method", y="Average Profit", hue="Dataset", data=avg_df_melted)

    # 加上註解
    for patch, (method, ds) in zip(
        ax.patches, zip(avg_df_melted["Method"], avg_df_melted["Dataset"])
    ):
        if ds == "Train":
            pct_base = avg_df.loc[avg_df.Method == method, "Train_%_Base"].values[0]
            pct_theory = avg_df.loc[avg_df.Method == method, "Train_%_Theory"].values[0]
        else:
            pct_base = avg_df.loc[avg_df.Method == method, "Test_%_Base"].values[0]
            pct_theory = avg_df.loc[avg_df.Method == method, "Test_%_Theory"].values[0]

        ax.annotate(
            f"{pct_base:.1f}%\n({pct_theory:.1f}%)",
            (patch.get_x() + patch.get_width() / 2, patch.get_height()),
            ha="center",
            va="bottom",
            fontsize=9,
            xytext=(0, 5),
            textcoords="offset points",
        )

    plt.title(f"Average Profit: {key}\n% Change vs Baseline / Theory Best")
    plt.ylabel("Average Profit")
    plt.xlabel("Method")
    plt.xticks(rotation=30)
    plt.legend(title="Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()

    # 儲存圖片
    plot_filename = os.path.join(output_dir, f"avg_profit_{key.replace('.csv','')}.png")
    plt.savefig(plot_filename)
    plt.close()

    print(f"✅ Saved plot for: {key}")

print(f"\n🎉 All plots saved in: {output_dir}")

Found 4 matched pairs.
-----------------------------------
key: chunk20_lasso1_fold5_lst0.8.csv
train_files[key]: train_chunk20_lasso1_fold5_lst0.8.csv
test_files[key]: test_chunk20_lasso1_fold5_lst0.8.csv
✅ Saved plot for: chunk20_lasso1_fold5_lst0.8.csv
-----------------------------------
key: chunk20_lasso1_fold5_lst0.9.csv
train_files[key]: train_chunk20_lasso1_fold5_lst0.9.csv
test_files[key]: test_chunk20_lasso1_fold5_lst0.9.csv
✅ Saved plot for: chunk20_lasso1_fold5_lst0.9.csv
-----------------------------------
key: chunk30_lasso1_fold5_lst0.8.csv
train_files[key]: train_chunk30_lasso1_fold5_lst0.8.csv
test_files[key]: test_chunk30_lasso1_fold5_lst0.8.csv
✅ Saved plot for: chunk30_lasso1_fold5_lst0.8.csv
-----------------------------------
key: chunk30_lasso1_fold5_lst0.9.csv
train_files[key]: train_chunk30_lasso1_fold5_lst0.9.csv
test_files[key]: test_chunk30_lasso1_fold5_lst0.9.csv
✅ Saved plot for: chunk30_lasso1_fold5_lst0.9.csv

🎉 All plots saved in: /Users/hanyuan/Github/

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 設定 seaborn 樣式
sns.set(style="whitegrid")

# 🔧 參數設定
input_dir = "/Users/hanyuan/Github/Two-Phase-Newsvendor/results_0407"
output_dir = os.path.join(input_dir, "plots")
os.makedirs(output_dir, exist_ok=True)  # 如果 plots 資料夾不存在就建立

# 🔍 取得所有 train/test 檔案
files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]
train_files = {f.replace("train_", ""): f for f in files if f.startswith("train_")}
test_files = {f.replace("test_", ""): f for f in files if f.startswith("test_")}

# 🔗 找出交集組合鍵
common_keys = sorted(set(train_files.keys()) & set(test_files.keys()))

print(f"Found {len(common_keys)} matched pairs.")

# 🎯 對每組配對做分析和畫圖
for key in common_keys:
    print(f"-----------------------------------")
    print(f"key: {key}")
    print(f"train_files[key]: {train_files[key]}")
    print(f"test_files[key]: {test_files[key]}")

    train_path = os.path.join(input_dir, train_files[key])
    test_path = os.path.join(input_dir, test_files[key])

    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    # train_df = train_df.drop(columns=["S12", "S15"])

    # ========== 1️⃣ 計算 Relative (%) vs Baseline ==========
    baseline_train = train_df["baseline"].to_numpy().reshape(-1, 1)
    baseline_test = test_df["baseline"].to_numpy().reshape(-1, 1)

    train_relative = (train_df.to_numpy() - baseline_train) / baseline_train * 100
    test_relative = (test_df.to_numpy() - baseline_test) / baseline_test * 100

    train_relative = pd.DataFrame(
        train_relative, columns=train_df.columns, index=train_df.index
    )
    test_relative = pd.DataFrame(
        test_relative, columns=test_df.columns, index=test_df.index
    )

    train_relative["Fold"] = train_relative.index + 1
    test_relative["Fold"] = test_relative.index + 1

    train_long = train_relative.melt(
        id_vars="Fold", var_name="Method", value_name="Relative Profit (%)"
    )
    test_long = test_relative.melt(
        id_vars="Fold", var_name="Method", value_name="Relative Profit (%)"
    )
    train_long["Dataset"] = "Train"
    test_long["Dataset"] = "Test"

    fold_long = pd.concat([train_long, test_long], axis=0)
    fold_long = fold_long.reset_index(drop=True)

    # 🎨 1. Line Plot
    plt.figure(figsize=(12, 6))
    sns.lineplot(
        data=fold_long,
        x="Fold",
        y="Relative Profit (%)",
        hue="Method",
        style="Dataset",
        markers=True,
        dashes=False,
    )
    plt.axhline(0, color="gray", linestyle="--", linewidth=1)
    plt.title(f"[{key}] Strategy Performance Across Folds (Relative to Baseline)")
    plt.legend(title="Method & Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"line_{key.replace('.csv','')}.png"))
    plt.close()

    # 🎨 2. Box Plot
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=fold_long, x="Method", y="Relative Profit (%)", hue="Dataset")
    plt.axhline(0, color="gray", linestyle="--", linewidth=1)
    plt.title(f"[{key}] Strategy Performance Distribution Across Folds")
    plt.xticks(rotation=30)
    plt.legend(title="Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"box_{key.replace('.csv','')}.png"))
    plt.close()

    # ========== 2️⃣ Heatmap: Relative vs Theory ==========
    theory_best_train = train_df["S14"].to_numpy().reshape(-1, 1)
    theory_best_test = test_df["S14"].to_numpy().reshape(-1, 1)

    train_theory_rel = (
        (train_df.to_numpy() - theory_best_train) / theory_best_train * 100
    )
    test_theory_rel = (test_df.to_numpy() - theory_best_test) / theory_best_test * 100

    train_theory_df = pd.DataFrame(
        train_theory_rel, columns=train_df.columns, index=train_df.index
    )
    test_theory_df = pd.DataFrame(
        test_theory_rel, columns=test_df.columns, index=test_df.index
    )

    train_theory_df["Fold"] = train_theory_df.index + 1
    test_theory_df["Fold"] = test_theory_df.index + 1

    train_theory_long = train_theory_df.melt(
        id_vars="Fold", var_name="Method", value_name="Relative vs Theory (%)"
    )
    train_theory_long["Dataset"] = "Train"
    test_theory_long = test_theory_df.melt(
        id_vars="Fold", var_name="Method", value_name="Relative vs Theory (%)"
    )
    test_theory_long["Dataset"] = "Test"

    merged = fold_long.merge(
        pd.concat([train_theory_long, test_theory_long], axis=0),
        on=["Fold", "Method", "Dataset"],
    )

    heatmap_data = merged.pivot(
        index="Fold", columns=["Method", "Dataset"], values="Relative Profit (%)"
    )
    annot = merged.assign(
        annot=merged["Relative Profit (%)"].round(1).astype(str)
        + "\n("
        + merged["Relative vs Theory (%)"].round(1).astype(str)
        + "%)"
    ).pivot(index="Fold", columns=["Method", "Dataset"], values="annot")

    # 🎨 3. Heatmap
    plt.figure(figsize=(14, 8))
    sns.heatmap(heatmap_data, annot=annot, fmt="", cmap="coolwarm", linewidths=0.5)
    plt.title(
        f"[{key}] Relative Profit (%) Across Folds\n(vs Baseline / (vs Theory Best))"
    )
    plt.ylabel("Fold")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"heatmap_{key.replace('.csv','')}.png"))
    plt.close()

    print(f"✅ Plots saved for: {key}")

print(f"\n🎉 All plots saved to: {output_dir}")

Found 4 matched pairs.
-----------------------------------
key: chunk20_lasso1_fold5_lst0.8.csv
train_files[key]: train_chunk20_lasso1_fold5_lst0.8.csv
test_files[key]: test_chunk20_lasso1_fold5_lst0.8.csv
✅ Plots saved for: chunk20_lasso1_fold5_lst0.8.csv
-----------------------------------
key: chunk20_lasso1_fold5_lst0.9.csv
train_files[key]: train_chunk20_lasso1_fold5_lst0.9.csv
test_files[key]: test_chunk20_lasso1_fold5_lst0.9.csv
✅ Plots saved for: chunk20_lasso1_fold5_lst0.9.csv
-----------------------------------
key: chunk30_lasso1_fold5_lst0.8.csv
train_files[key]: train_chunk30_lasso1_fold5_lst0.8.csv
test_files[key]: test_chunk30_lasso1_fold5_lst0.8.csv
✅ Plots saved for: chunk30_lasso1_fold5_lst0.8.csv
-----------------------------------
key: chunk30_lasso1_fold5_lst0.9.csv
train_files[key]: train_chunk30_lasso1_fold5_lst0.9.csv
test_files[key]: test_chunk30_lasso1_fold5_lst0.9.csv
✅ Plots saved for: chunk30_lasso1_fold5_lst0.9.csv

🎉 All plots saved to: /Users/hanyuan/Git

In [5]:
# # 1️⃣ 計算平均 profit
# train_means = train_all_fold_profit_df.mean()
# test_means = test_all_fold_profit_df.mean()

# # 2️⃣ 定義 baseline & theory best
# baseline_train = train_means["baseline"]
# baseline_test = test_means["baseline"]
# theory_best_train = train_means["S14"]
# theory_best_test = test_means["S14"]

# # 3️⃣ 計算百分比變化：baseline & theory
# train_pct_base = (train_means - baseline_train) / baseline_train * 100
# test_pct_base = (test_means - baseline_test) / baseline_test * 100
# train_pct_theory = (train_means - theory_best_train) / theory_best_train * 100
# test_pct_theory = (test_means - theory_best_test) / theory_best_test * 100

# # 4️⃣ 建 DataFrame
# avg_df = pd.DataFrame(
#     {
#         "Method": train_means.index,
#         "Train": train_means.values,
#         "Test": test_means.values,
#         "Train_%_Base": train_pct_base.values,
#         "Test_%_Base": test_pct_base.values,
#         "Train_%_Theory": train_pct_theory.values,
#         "Test_%_Theory": test_pct_theory.values,
#     }
# )

# avg_df_melted = avg_df.melt(
#     id_vars=[
#         "Method",
#         "Train_%_Base",
#         "Test_%_Base",
#         "Train_%_Theory",
#         "Test_%_Theory",
#     ],
#     value_vars=["Train", "Test"],
#     var_name="Dataset",
#     value_name="Average Profit",
# )

# # 5️⃣ 畫圖
# plt.figure(figsize=(15, 9))
# ax = sns.barplot(x="Method", y="Average Profit", hue="Dataset", data=avg_df_melted)

# # 6️⃣ 標註：baseline (%) 在第一行、theory (%) 括號內第二行
# for patch, (method, ds) in zip(
#     ax.patches, zip(avg_df_melted["Method"], avg_df_melted["Dataset"])
# ):
#     if ds == "Train":
#         pct_base = avg_df.loc[avg_df.Method == method, "Train_%_Base"].values[0]
#         pct_theory = avg_df.loc[avg_df.Method == method, "Train_%_Theory"].values[0]
#     else:
#         pct_base = avg_df.loc[avg_df.Method == method, "Test_%_Base"].values[0]
#         pct_theory = avg_df.loc[avg_df.Method == method, "Test_%_Theory"].values[0]

#     ax.annotate(
#         f"{pct_base:.1f}%\n({pct_theory:.1f}%)",
#         (patch.get_x() + patch.get_width() / 2, patch.get_height()),
#         ha="center",
#         va="bottom",
#         fontsize=9,
#         xytext=(0, 5),
#         textcoords="offset points",
#     )

# plt.title("Average Profit (Train vs Test) — % Change vs Baseline / Theory Best")
# plt.ylabel("Average Profit")
# plt.xlabel("Method")
# plt.xticks(rotation=30)
# plt.legend(title="Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
# plt.tight_layout()
# plt.show()

In [6]:
# # 計算 baseline（訓練與測試）
# baseline_train = train_all_fold_profit_df["baseline"].to_numpy().reshape(-1, 1)
# baseline_test = test_all_fold_profit_df["baseline"].to_numpy().reshape(-1, 1)

# # 計算百分比變化
# train_relative = (
#     (train_all_fold_profit_df.to_numpy() - baseline_train) / baseline_train * 100
# )
# test_relative = (
#     (test_all_fold_profit_df.to_numpy() - baseline_test) / baseline_test * 100
# )

# # 轉回 DataFrame，並保留 column names
# train_relative = pd.DataFrame(
#     train_relative,
#     columns=train_all_fold_profit_df.columns,
#     index=train_all_fold_profit_df.index,
# )
# test_relative = pd.DataFrame(
#     test_relative,
#     columns=test_all_fold_profit_df.columns,
#     index=test_all_fold_profit_df.index,
# )

# # 加入 fold 編號
# train_relative["Fold"] = train_relative.index + 1
# test_relative["Fold"] = test_relative.index + 1

# # 轉換成長格式
# train_long = train_relative.melt(
#     id_vars="Fold", var_name="Method", value_name="Relative Profit (%)"
# )
# train_long["Dataset"] = "Train"

# test_long = test_relative.melt(
#     id_vars="Fold", var_name="Method", value_name="Relative Profit (%)"
# )
# test_long["Dataset"] = "Test"

# # 合併數據
# fold_long = pd.concat([train_long, test_long], axis=0)

# # === 1. 使用線圖 (Line Plot) 觀察不同 Fold 上的變化趨勢 ===
# plt.figure(figsize=(12, 6))
# sns.lineplot(
#     data=fold_long,
#     x="Fold",
#     y="Relative Profit (%)",
#     hue="Method",
#     style="Dataset",
#     markers=True,
#     dashes=False,
# )
# plt.axhline(0, color="gray", linestyle="--", linewidth=1)  # 基準線
# plt.title("Strategy Performance Across Folds (Relative to Baseline)")
# plt.legend(title="Method & Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
# plt.show()

# # === 2. 使用箱型圖 (Box Plot) 查看策略穩定性 ===
# plt.figure(figsize=(12, 6))
# sns.boxplot(data=fold_long, x="Method", y="Relative Profit (%)", hue="Dataset")
# plt.axhline(0, color="gray", linestyle="--", linewidth=1)
# plt.title("Strategy Performance Distribution Across Folds")
# plt.xticks(rotation=30)
# plt.legend(title="Dataset", bbox_to_anchor=(1.05, 1), loc="upper left")
# plt.show()

# # 3️⃣ Heatmap：同時顯示 vs Baseline & vs Theory Best（每 Fold 的 S14）
# theory_best_train = train_all_fold_profit_df["S14"].to_numpy().reshape(-1, 1)
# theory_best_test = test_all_fold_profit_df["S14"].to_numpy().reshape(-1, 1)

# # vs Theory (%) 計算
# train_theory_rel = (
#     (train_all_fold_profit_df.to_numpy() - theory_best_train) / theory_best_train * 100
# )
# test_theory_rel = (
#     (test_all_fold_profit_df.to_numpy() - theory_best_test) / theory_best_test * 100
# )

# # 回 DataFrame 並 melt
# train_theory_rel = pd.DataFrame(
#     train_theory_rel,
#     columns=train_all_fold_profit_df.columns,
#     index=train_all_fold_profit_df.index,
# )
# train_theory_rel["Fold"] = train_theory_rel.index + 1
# train_theory_long = train_theory_rel.melt(
#     id_vars="Fold", var_name="Method", value_name="Relative vs Theory (%)"
# )
# train_theory_long["Dataset"] = "Train"

# test_theory_rel = pd.DataFrame(
#     test_theory_rel,
#     columns=test_all_fold_profit_df.columns,
#     index=test_all_fold_profit_df.index,
# )
# test_theory_rel["Fold"] = test_theory_rel.index + 1
# test_theory_long = test_theory_rel.melt(
#     id_vars="Fold", var_name="Method", value_name="Relative vs Theory (%)"
# )
# test_theory_long["Dataset"] = "Test"

# # 合併 baseline (%) 與 theory (%) 資料
# merged = fold_long.merge(
#     pd.concat([train_theory_long, test_theory_long], axis=0),
#     on=["Fold", "Method", "Dataset"],
# )

# # Pivot heatmap values + annotations
# heatmap_data = merged.pivot(
#     index="Fold", columns=["Method", "Dataset"], values="Relative Profit (%)"
# )
# annot = merged.assign(
#     annot=merged["Relative Profit (%)"].round(1).astype(str)
#     + "\n("
#     + merged["Relative vs Theory (%)"].round(1).astype(str)
#     + "%)"
# ).pivot(index="Fold", columns=["Method", "Dataset"], values="annot")

# plt.figure(figsize=(14, 8))
# sns.heatmap(heatmap_data, annot=annot, fmt="", cmap="coolwarm", linewidths=0.5)
# plt.title("Relative Profit (%) Across Folds\n(vs Baseline / (vs Theory Best))")
# plt.ylabel("Fold")
# plt.tight_layout()
# plt.show()

In [7]:
# # 訓練階段分佈

# baseline_data = []
# S1_data = []
# S2_data = []
# S12_data = []
# S14_data = []
# S15_data = []

# for result in train_all_fold_stimulation_results:
#     baseline_data.append(result["baseline"])
#     S1_data.append(result["S1"])
#     S2_data.append(result["S2"])
#     S12_data.append(result["S12"])
#     S14_data.append(result["S14"])
#     S15_data.append(result["S15"])

# # 合併數據
# baseline_df = pd.concat(baseline_data, ignore_index=True)
# S1_df = pd.concat(S1_data, ignore_index=True)
# S2_df = pd.concat(S2_data, ignore_index=True)
# S12_df = pd.concat(S12_data, ignore_index=True)
# S14_df = pd.concat(S14_data, ignore_index=True)
# S15_df = pd.concat(S15_data, ignore_index=True)


# dfs = {
#     "baseline": baseline_df,
#     "S1": S1_df,
#     "S2": S2_df,
#     "S12": S12_df,
#     "S15": S15_df,
#     "S14": S14_df,
# }

# # 調用繪圖函數
# plot_strategies_profits_scatter(f"{status}_{model_prefix}", dfs)

In [8]:
# # 測試結果分布圖

# baseline_data = []
# S1_data = []
# S2_data = []
# S12_data = []
# S14_data = []
# S15_data = []

# for result in test_all_fold_stimulation_results:
#     baseline_data.append(result["baseline"])
#     S1_data.append(result["S1"])
#     S2_data.append(result["S2"])
#     S12_data.append(result["S12"])
#     S14_data.append(result["S14"])
#     S15_data.append(result["S15"])
# # 合併數據
# baseline_df = pd.concat(baseline_data, ignore_index=True)
# S1_df = pd.concat(S1_data, ignore_index=True)
# S2_df = pd.concat(S2_data, ignore_index=True)
# S12_df = pd.concat(S12_data, ignore_index=True)
# S14_df = pd.concat(S14_data, ignore_index=True)
# S15_df = pd.concat(S15_data, ignore_index=True)

# dfs = {
#     "baseline": baseline_df,
#     "S1": S1_df,
#     "S2": S2_df,
#     "S12": S12_df,
#     "S15": S15_df,
#     "S14": S14_df,
# }

# # 調用繪圖函數
# plot_strategies_profits_scatter(f"{status}_{model_prefix}", dfs)